In [1]:
import torch
import torch.nn as nn
from a0_alexNet import AlexNet,model_partition
device = "cuda" if torch.cuda.is_available() else "cpu"

alexnet = AlexNet(input_layer=3,num_classes=1000)
alexnet = alexnet.to(device)


set_index = 0
edge_model,cloud_model = model_partition(alexnet,set_index)
print(f"alexnet model : {len(alexnet)}")
print(f"------------- edge model -----------------")
print(edge_model)
print(f"------------- cloud model -----------------")
print(cloud_model)

alexnet model : 22
------------- edge model -----------------
Sequential()
------------- cloud model -----------------
Sequential(
  (1-Conv2d): Conv2d(3, 64, kernel_size=(11, 11), stride=(4, 4), padding=(2, 2))
  (2-ReLU): ReLU(inplace=True)
  (3-MaxPool2d): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (4-Conv2d): Conv2d(64, 192, kernel_size=(5, 5), stride=(1, 1), padding=(2, 2))
  (5-ReLU): ReLU(inplace=True)
  (6-MaxPool2d): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (7-Conv2d): Conv2d(192, 384, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8-ReLU): ReLU(inplace=True)
  (9-Conv2d): Conv2d(384, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (10-ReLU): ReLU(inplace=True)
  (11-Conv2d): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (12-ReLU): ReLU(inplace=True)
  (13-MaxPool2d): MaxPool2d(kernel_size=3, stride=2, padding=0, dilation=1, ceil_mode=False)
  (14-AdaptiveAvgPool2d): 

In [4]:
"""edge_cloud运行在边缘设备上 计算结果"""
import time
x = torch.rand(size=(10000,3,224,224))
x = x.to(device)
print(f"x device : {x.device}")

if len(edge_model) > 0:
    print(f"model device : {next(edge_model.parameters()).device}")
    for layer in edge_model:
        start_time = int(round(time.time() * 1000))
        x = layer(x)
        end_time = int(round(time.time() * 1000))
        # print(x.device)
        print(layer.__class__.__name__,'output shape:\t',x.shape,
              f'\t computation time: {(end_time - start_time)/1000 :>3f} s')

x device : cpu


In [6]:
print(f"x device : {x.device}")

if len(cloud_model) > 0:
    print(f"model device : {next(cloud_model.parameters()).device}")
    for layer in cloud_model:
        start_time = int(round(time.time() * 1000))
        x = layer(x)
        end_time = int(round(time.time() * 1000))
        # print(x.device)
        print(layer.__class__.__name__,'output shape:\t',x.shape,
              f'\t computation time: {(end_time - start_time)/1000 :>3f} s')

model device : cpu
x device : cpu
ReLU output shape:	 torch.Size([10000, 192, 27, 27]) 	 computation time: 0.694000 s
MaxPool2d output shape:	 torch.Size([10000, 192, 13, 13]) 	 computation time: 2.449000 s
Conv2d output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 7.234000 s
ReLU output shape:	 torch.Size([10000, 384, 13, 13]) 	 computation time: 0.317000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 7.296000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 0.145000 s
Conv2d output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 6.849000 s
ReLU output shape:	 torch.Size([10000, 256, 13, 13]) 	 computation time: 0.210000 s
MaxPool2d output shape:	 torch.Size([10000, 256, 6, 6]) 	 computation time: 0.820000 s
AdaptiveAvgPool2d output shape:	 torch.Size([10000, 256, 6, 6]) 	 computation time: 0.287000 s
Flatten output shape:	 torch.Size([10000, 9216]) 	 computation time: 0.011000 s
Dropout output shape: